## Markov chain

In [1]:
import os
import random
import numpy as np
from collections import defaultdict, Counter
from miditoolkit import MidiFile, Instrument, Note
import math

In [ ]:
# MIDI specs
TICKS_PER_BEAT = 22050
# quantizations
DURATION_VALUES = [int(TICKS_PER_BEAT * d) for d in [0.25, 0.5, 0.75, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0]]

In [3]:
def quantize_duration(val, choices):
    return min(choices, key=lambda x: abs(x - val))

In [4]:
def quantize_velocity(velocity):
    if velocity == 0: return 0
    elif velocity < 4: return 2
    elif velocity < 8: return 4
    elif velocity < 12: return 6
    elif velocity < 16: return 8
    else: return 10

In [5]:
def extract_tokens_by_instrument(midi_path):
    midi = MidiFile(midi_path)
    program_tokens = defaultdict(list)
    for track in midi.instruments:
        notes = sorted(track.notes, key=lambda n: n.start)
        current_time = 0
        for note in notes:
            if note.start > current_time:
                rest_dur = quantize_duration(note.start - current_time, DURATION_VALUES)
                program_tokens[track.program].append((0, rest_dur, 0))  # rest
            pitch = note.pitch
            dur = quantize_duration(note.end - note.start, DURATION_VALUES)
            vel = quantize_velocity(note.velocity)
            program_tokens[track.program].append((pitch, dur, vel))
            current_time = note.end
    return program_tokens

In [ ]:
# constructs a unigram chain for a single instrument
def build_markov_chain(tokens):
    transitions = defaultdict(Counter)
    for i in range(len(tokens) - 1):
        transitions[tokens[i]][tokens[i + 1]] += 1
    return transitions

In [ ]:
# builds and returns a chain for each instrument
def build_chains_for_all(mididir):
    all_program_chains = defaultdict(list)
    for file in os.listdir(mididir):
        if file.endswith(".mid"):
            full_path = os.path.join(mididir, file)
            prog_tokens = extract_tokens_by_instrument(full_path)
            for program, tokens in prog_tokens.items():
                all_program_chains[program].extend(tokens)
    return {program: build_markov_chain(tokens) for program, tokens in all_program_chains.items()}

In [ ]:
# take an action
def sample_next(transitions, current_token):
    next_tokens = transitions.get(current_token)
    if not next_tokens:
        return random.choice(list(transitions.keys()))
    total = sum(next_tokens.values())
    choices, weights = zip(*next_tokens.items())
    probs = [w / total for w in weights]
    return random.choices(choices, probs)[0]

In [ ]:
# samples for specified number of times
def generate_sequence(chain, length=200):
    current = random.choice(list(chain.keys()))
    sequence = [current]
    for _ in range(length - 1):
        current = sample_next(chain, current)
        sequence.append(current)
    return sequence

In [ ]:
# translational variables
NAME_TO_PROGRAM = {
    'p1': 80,
    'p2': 81,
    'tr': 38,
    'no': 121
}

PROGRAM_TO_NAME = {
    80: 'p1',
    81: 'p2',
    38: 'tr',
    121: 'no'
}

In [ ]:
# merge tracks into one MIDI file
def save_multi_program_midi(program_sequences, filename, ticks_per_beat=TICKS_PER_BEAT):
    midi = MidiFile(ticks_per_beat=TICKS_PER_BEAT)
    for program, tokens in program_sequences.items():
        track = Instrument(program=program, is_drum=False, name=PROGRAM_TO_NAME[program])
        time = 0
        for pitch, dur, vel in tokens:
            if pitch == 0:
                time += dur  # rest
            else:
                note = Note(velocity=vel, pitch=pitch, start=time, end=time + dur)
                track.notes.append(note)
                time += dur
        midi.instruments.append(track)
    midi.dump(filename)
    return midi

In [12]:
MIDI_DIR = "nesmdb_midi/train"

In [13]:
chains = build_chains_for_all(MIDI_DIR)

## Sequence generation and log-likelihood

In [ ]:
# unlike hmmlearn HMM which has a built in log-likelihood scorer,
# we need to score the chain ourselves
def compute_markov_log_likelihood(sequence, chain, smoothing=1e-8):
    # convert counts to probabilities
    transition_probs = {
        state: {
            next_state: count / sum(next_states.values())
            for next_state, count in next_states.items()
        }
        for state, next_states in chain.items()
    }

    log_likelihood = 0.0
    for i in range(len(sequence) - 1):
        s1, s2 = sequence[i], sequence[i + 1]
        prob = transition_probs.get(s1, {}).get(s2, smoothing)  # Smoothing for unseen transitions
        log_likelihood += math.log(prob)
    return log_likelihood

In [ ]:
# normalize based on the length of a generated sequence
def normalized_log_likelihood(sequence, chain):
    return compute_markov_log_likelihood(sequence, chain) / max(1, len(sequence))

In [55]:
generated_sequences = {
    prog: generate_sequence(chain, length=400) for prog, chain in chains.items()
}

In [56]:
for prog, sequence in generated_sequences.items():
    chain = chains[prog]
    print(PROGRAM_TO_NAME[prog])
    print("Normalized log-likelihood:", normalized_log_likelihood(sequence, chain))

p1
Normalized log-likelihood: -3.819025192351291
tr
Normalized log-likelihood: -2.6025356194206015
p2
Normalized log-likelihood: -3.6414802107887425
no
Normalized log-likelihood: -2.4549657032047305


In [57]:
out = save_multi_program_midi(generated_sequences, "outputs/markov_19.mid")